In [5]:
import os
import yaml
import glob
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import wandb
from wandb.integration.keras import WandbCallback

In [6]:
wandb.login(key=017feefe0af6702cda76aab121ec71cf3a362fec)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (847546409.py, line 1)

In [7]:
dataset_yaml = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml"

In [17]:
def build_detector(input_shape=(128, 128, 3), output_dim=5):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(output_dim, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

In [9]:
def load_yolo_dataset(image_dir, label_dir, img_size=(128, 128), max_objects=1):
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
    x_data, y_data = [], []

    for img_path in image_paths:
        filename = os.path.basename(img_path).replace('.jpg', '')
        label_path = os.path.join(label_dir, f"{filename}.txt")

        # Load and resize image
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0  # normalize
        x_data.append(img)

        # Load YOLO label
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()

            # Take first object only for simplicity
            if lines:
                parts = list(map(float, lines[0].strip().split()))
                class_id, x, y, w, h = parts
                y_data.append([x, y, w, h, class_id])
            else:
                y_data.append([0, 0, 0, 0, 0])
        else:
            y_data.append([0, 0, 0, 0, 0])  # empty box

    return np.array(x_data, dtype=np.float32), np.array(y_data, dtype=np.float32)


In [13]:
with open(dataset_yaml, "r") as f:
    config = yaml.safe_load(f)

train_img_dir = config["train"]
val_img_dir = config["val"]
test_img_dir = config.get("test", None) 
class_names = config["names"]
#num_classes = config["nc"]

In [14]:
x_train, y_train = load_yolo_dataset(train_img_dir, train_img_dir.replace('images', 'labels'))
x_val, y_val = load_yolo_dataset(val_img_dir, val_img_dir.replace('images', 'labels'))

In [15]:
wandb.init(project="Own CNN", name="Detector", config=config)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: samutheman (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [18]:
model = build_detector()
model.compile(optimizer='adam', loss='mse')  # simple loss for now

model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=20,
    batch_size=16,
    callbacks=[WandbCallback()]
)

2025-05-01 11:43:57.045356: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13294 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:29:00.0, compute capability: 8.6
wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/20


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(16,), dtype=float32). Expected shape (None, 128, 128, 3), but input has incompatible shape (16,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(16,), dtype=float32)
  • training=True
  • mask=None